# Лабораторна робота №2
### "Розв’язання систем лінійних алгебраїчних рівнянь"
### Самарука Владислав Володимировича


## 1. Написати програму, яка розв’язує систему лінійних алгебраїчних рівнянь двома методами:

#### 4) Методом квадратних коренів
#### 7) Методом Зейделя

## 2. Знайти визначник матриці своїм прямим методом. Знайти число обумовленості.



# Завдання 1

#### 4) Метод квадратних коренів

In [130]:
import numpy as np

A = np.array([
    [1, 2, 3, 4, 5],
    [6, 7, 8, 9, 10],
    [2, 1, 3, 5, 4],
    [8, 7, 6, 9, 10],
    [9, 6, 7, 10, 9]
])

b = np.array([1, 2, 3, 4, 5])

In [131]:
def check_symetric(A) -> bool:
    return np.allclose(A, A.T)

print(check_symetric(A))

False


In [132]:
def check_singular(A) -> bool:
    return np.linalg.det(A) != 0

print(check_singular(A))

True


##### A = S.T * D * S

In [133]:
def calculate_S_and_D(A):
    S = np.zeros(A.shape)
    D = np.zeros(A.shape)
    D[0, 0] = np.sign(A[0, 0])
    S[0, 0] = np.sqrt(np.abs(A[0, 0]))

    for i in range(0, A.shape[0]):
        for j in range(1, A.shape[1]):
            if i == j:
                suma = 0
                for p in range(j):
                    suma += S[p, j] ** 2 * D[p, p]
                
                D[j, j] = np.sign(A[j, j] - suma)
                S[j, j] = np.sqrt(np.abs(A[j, j] - suma))

            elif j > i:
                suma = 0
                for p in range(i):
                    suma += S[p, i] * D[p, p] * S[p, j]
                
                S[i, j] = (A[i, j] - suma) / (S[i, i] * D[i, i])
                    
                
    return S, D

S, D = calculate_S_and_D(A)
print("S:")
print(S)
print("D:")
print(D)

S:
[[1.         2.         3.         4.         5.        ]
 [0.         1.73205081 1.15470054 0.57735027 0.        ]
 [0.         0.         2.7080128  2.83110429 4.0620192 ]
 [0.         0.         0.         0.82572282 1.81659021]
 [0.         0.         0.         0.         1.67332005]]
D:
[[ 1.  0.  0.  0.  0.]
 [ 0.  1.  0.  0.  0.]
 [ 0.  0. -1.  0.  0.]
 [ 0.  0.  0.  1.  0.]
 [ 0.  0.  0.  0. -1.]]


##### Розв’язок системи рiвнянь S.T * D * y = b та S * x = y.

In [134]:
def solve_equation_in_reverse(A, b):
    x = np.zeros(b.shape)
    for i in range(A.shape[0] - 1, -1, -1):
        suma = 0
        for j in range(i + 1, A.shape[1]):
            suma += A[i, j] * x[j]
        x[i] = (b[i] - suma) / A[i, i]

    return x

def solve_equation_in_forward(A, b):
    x = np.zeros(b.shape)
    for i in range(A.shape[0]):
        suma = 0
        for j in range(i):
            suma += A[i, j] * x[j]
        x[i] = (b[i] - suma) / A[i, i]

    return x

STD = S.T @ D
print("STD:")
print(STD)
print("b:")
print(b)
y = solve_equation_in_forward(STD, b)
print("y:")
print(y)

STD:
[[ 1.          0.          0.          0.          0.        ]
 [ 2.          1.73205081  0.          0.          0.        ]
 [ 3.          1.15470054 -2.7080128   0.          0.        ]
 [ 4.          0.57735027 -2.83110429  0.82572282  0.        ]
 [ 5.          0.         -4.0620192   1.81659021 -1.67332005]]
b:
[1 2 3 4 5]
y:
[ 1.  0. -0.  0. -0.]


In [135]:
x = solve_equation_in_reverse(S, y)
print("x:")
print(x)

x:
[ 1.  0. -0.  0. -0.]


#### 7) Метод Зейделя

In [136]:
A = np.array([
    [12, 3, 5, 8, 9],
    [3, 15, 7, 6, 4],
    [5, 7, 20, 11, 14],
    [8, 6, 11, 25, 10],
    [9, 4, 14, 10, 30]
])

b = np.array([1, 2, 3, 4, 5])

##### Перевірки умов збіжності:

In [137]:
symetric = check_symetric(A)

def calculate_minors_determinants(A):
    minor_det = []
    for i in range(A.shape[0]):
        minor_det.append(np.linalg.det(A[:i + 1, :i + 1]))

    return minor_det    

minor_det = calculate_minors_determinants(A)
if symetric and np.all(np.array(minor_det) > 0):
    print("Збігається")
else:
    print("Не збігається")


Збігається


#### Вирішення

In [138]:
eps = 0.01

def calculate_next_x(A, b, x) -> np.array:
    next_x = np.zeros(x.shape)
    for i in range(A.shape[0]):
        sum1 = 0
        for j in range(i):
            sum1 += A[i, j]/A[i, i] * next_x[j]
        sum2 = 0
        for j in range(i + 1, A.shape[0]):
            sum2 += A[i, j]/A[i, i] * x[j]
        
        next_x[i] = -sum1 - sum2 + b[i]/A[i, i]
    
    return next_x

def calculate_norm(x, next_x) -> float:
    return np.linalg.norm(x - next_x, ord=np.inf)

def solve_equation(A, b, eps):
    x = np.zeros(b.shape)
    next_x = calculate_next_x(A, b, x)
    print("next_x: ")
    print(next_x)
    while True:
        print("x:")
        print(x)
        print("norma:")
        print(calculate_norm(x, next_x))

        if calculate_norm(x, next_x) < eps:
            break

        x = next_x
        next_x = calculate_next_x(A, b, x)

    return next_x

x = solve_equation(A, b, eps)
print("x:")
print(x)

next_x: 
[0.08333333 0.11666667 0.08833333 0.06646667 0.06273333]
x:
[0. 0. 0. 0. 0.]
norma:
0.11666666666666667
x:
[0.08333333 0.11666667 0.08833333 0.06646667 0.06273333]
norma:
0.15733333333333333
x:
[-0.074       0.06359556  0.06577156  0.11438425  0.11156578]
norma:
0.04590087407407406
x:
[-0.11990087  0.05111554  0.02107739  0.13220018  0.14191868]
norma:
0.03069677892753997
x:
[-0.13280026  0.05933221 -0.00961939  0.13572141  0.15784436]
norma:
0.015479084300969004
x:
[-0.13635585  0.06871316 -0.02509847  0.1350483   0.16510819]
norma:
0.006497942359824427
x:
[-0.1372506   0.0744479  -0.03159641  0.13391184  0.16802317]


### Завдання 2


##### Визначник матриці методом квадратних коренів:

In [139]:
def det(A):
    S, D = calculate_S_and_D(A)
    det = 1
    for i in range(A.shape[0]):
        det *= D[i, i] * S[i, i] ** 2

    return det

print("det:")
print(det(A))


det:
742583.9999999998


#### Число обумовленості

In [140]:
def condtion_number(A):
    return np.linalg.norm(A, ord=np.inf) * np.linalg.norm(np.linalg.inv(A), ord=np.inf)


print("Обернена матриця:")
print(np.linalg.inv(A))

print("Перша норма початкової матриці:")
print(np.linalg.norm(A, ord=1))

print("Друга норма початкової матриці:")
print(np.linalg.norm(A, ord=2))

print("Перша норма оберненої матриці:")
print(np.linalg.norm(np.linalg.inv(A), ord=1))

print("Друга норма оберненої матриці:")
print(np.linalg.norm(np.linalg.inv(A), ord=2))

print("condtion_number:")
print(condtion_number(A))


Обернена матриця:
[[ 0.12443037 -0.00953427  0.01018067 -0.02963705 -0.03092983]
 [-0.00953427  0.08244051 -0.02729523 -0.00757759  0.00713185]
 [ 0.01018067 -0.02729523  0.09552993 -0.02439455 -0.03586396]
 [-0.02963705 -0.00757759 -0.02439455  0.06175598  0.00070026]
 [-0.03092983  0.00713185 -0.03586396  0.00070026  0.05816446]]
Перша норма початкової матриці:
67.0
Друга норма початкової матриці:
55.33292934090252
Перша норма оберненої матриці:
0.2047121941760124
Друга норма оберненої матриці:
0.1653839583893026
condtion_number:
13.71571700979283
